In [1]:
#Import Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model

In [3]:
#Reading the data and viewing

In [4]:
df = pd.read_csv('/Users/srinathnarayanan/Downloads/Misc/DSD.csv',header = 0,index_col = False)

In [5]:
df

,CustStorItemTriadID,BaseorderID,Createdate,RecDeliveryDate,CategoryID,OaMasterDistributorID,ConversionFactor,InventoryChange,ActualScans,ForecastedScans,...,ConversionUnits,MaxDeliveredQty,ConversionResidual,Previous2DelDate,MaxScanDate,MaxShipDate,TwoOrderAgo,QtyShippedTwoOrderAgo,Reviewed,IncInAnom
0,13761530,1867926,5/1/2020,5/7/2020,1,5,0.75,12,0,13,...,9,27,1,4/30/2020,4/29/2020,4/27/2020,9,0,1,1
1,13761530,1889852,5/7/2020,5/11/2020,1,5,0.75,3,0,18,...,9,27,6,5/4/2020,5/5/2020,5/4/2020,18,13,1,1
2,13761530,1899415,5/8/2020,5/14/2020,1,5,0.75,5,0,11,...,9,27,0,5/7/2020,5/6/2020,5/4/2020,9,0,1,1
3,13761530,1953002,5/21/2020,5/25/2020,1,5,0.75,5,0,16,...,9,27,0,5/18/2020,5/19/2020,5/18/2020,18,12,1,1
4,13761530,2048503,6/11/2020,6/15/2020,1,5,0.75,6,0,15,...,9,27,5,6/8/2020,6/9/2020,6/8/2020,18,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495597,21016901,2249563,7/24/2020,7/27/2020,1,5,0.75,1,0,3,...,4,8,2,7/20/2020,7/20/2020,7/20/2020,4,4,1,1
495598,21016901,2262011,7/28/2020,7/30/2020,1,5,0.75,4,0,3,...,4,8,0,7/23/2020,7/26/2020,7/20/2020,0,0,1,1
495599,21016901,2294886,8/4/2020,8/6/2020,1,5,0.75,5,0,5,...,4,8,1,7/30/2020,8/2/2020,7/30/2020,0,4,1,1
495600,21016901,2319043,8/7/2020,8/10/2020,1,5,0.75,7,0,3,...,4,8,0,8/3/2020,8/4/2020,7/30/2020,8,0,1,1


In [6]:
#Replacing Nan to zeros and make sure we dont delete any rows

In [7]:
df = df.fillna(0)

In [8]:
#Checking if Nan are replaced with zeros

In [9]:
df.isnull().values.any()

False

In [10]:
# Function to assign value 0 if there is no match between OriginalPropOrderQty numbers with the sum(Actual+Forecasted+Weightdata)

In [11]:
df['Original'] = df['WeightData']+df['BaseOrder']+df['TrueUpAdjQty']

In [12]:
def Original(df):
        if df['OriginalPropOrderQty'] == df['Original']:
            return 1
        return 0
df['Original'] = df.apply(Original, axis=1)

In [13]:
df.Original.value_counts()

1    406068
0     89534
Name: Original, dtype: int64

In [14]:
# Function to assign value 0 if there is no match between PropOrderQty and OriginalPropOrderQty

In [15]:
def target(df):
        if df['OriginalPropOrderQty'] == df['PropOrderQty']:
            return 1
        return 0
df['target'] = df.apply(target, axis=1) 

In [16]:
df.target.value_counts()

1    382184
0    113418
Name: target, dtype: int64

In [17]:
def targetsize(df):
        if df['MaxDeliveredQty'] == df['PropOrderQty']:
            return 1
        return 0
df['targetsize'] = df.apply(targetsize, axis=1) 

In [18]:
df.targetsize.value_counts()

0    448096
1     47506
Name: targetsize, dtype: int64

In [19]:
X = df[['InventoryChange','ActualScans','ForecastedScans','SlowMoving','TrueUpAdjQty','TooBig','OriginalPropOrderQty','target','targetsize','Original']]
y = df['PropOrderQty']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [21]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [22]:
history = model.fit(X_train, y_train, batch_size=32, epochs=4, verbose=1, validation_split=0.2)

Epoch 1/4
158592/158592 [==============================] - 104s 652us/step - loss: 3.4373 - mean_squared_error: 3.4373 - val_loss: 1.3492 - val_mean_squared_error: 1.3492
Epoch 2/4
158592/158592 [==============================] - 104s 656us/step - loss: 1.2457 - mean_squared_error: 1.2457 - val_loss: 0.9566 - val_mean_squared_error: 0.9566
Epoch 3/4
158592/158592 [==============================] - 283s 2ms/step - loss: 1.1587 - mean_squared_error: 1.1587 - val_loss: 0.9533 - val_mean_squared_error: 0.9533
Epoch 4/4
158592/158592 [==============================] - 113s 711us/step - loss: 1.1568 - mean_squared_error: 1.1568 - val_loss: 0.9814 - val_mean_squared_error: 0.9814


In [23]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred)))

1.004585928400773
1.041965218305897
